In [1]:
import numpy as np
from datasets import list_datasets, load_dataset
from numpy import genfromtxt

In [2]:
import pandas as pd

In [3]:
from design.questions import *
from questions.prior_questions import *

In [4]:
questions = []
for i in range(len(g_movie_piror_keywords)):
    word = g_movie_piror_keywords[i]
    questions.append(generate_one_question(word, "n", "good"))
    questions.append(generate_one_question(word, "n", "bad"))
    #prior_question.extend(generate_questions(word, source="datamuse", appendix=None, max_question_num=20, question_type="good or bad"))

In [5]:
question_types = ["Yes-no" for _ in range(len(questions))]

In [6]:
questionnaire_data = pd.DataFrame({"Type":question_types, "Question": questions})

In [7]:
questionnaire_data

,Type,Question
0,Yes-no,is the text saying anything good about a film?
1,Yes-no,is the text saying anything bad about a film?
2,Yes-no,is the text saying anything good about a movie?
3,Yes-no,is the text saying anything bad about a movie?
4,Yes-no,is the text saying anything good about a proje...
...,...,...
73,Yes-no,is the text saying anything bad about a moviedom?
74,Yes-no,is the text saying anything good about a talkie?
75,Yes-no,is the text saying anything bad about a talkie?
76,Yes-no,is the text saying anything good about a script?


In [9]:
questionnaire_data.to_csv("questions/moviequestions.csv", index=False)

In [ ]:
X = genfromtxt('record/survey_rotten_tomatoes(train)_basicquestions_unifiedqa-t5-large.csv', delimiter=',')

In [ ]:
X.shape

In [ ]:
dataset = load_dataset('rotten_tomatoes', split='train')

In [ ]:
len(dataset)

In [ ]:
y = np.asarray([item['label'] for item in dataset])

In [ ]:
y.shape

In [ ]:
shuffler = np.random.permutation(len(X))
X = X[shuffler]
y = y[shuffler]


In [ ]:
X_train = X[:-1024]
y_train = y[:-1024]

X_test = X[-1024:]
y_test = y[-1024:]

In [ ]:
import xgboost as xgb

In [ ]:
# use DMatrix for xgbosot
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# set xgboost params
param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

In [ ]:
#------------- numpy array ------------------
# training and testing - numpy matrices
bst = xgb.train(param, dtrain, num_round)

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
# extracting most confident predictions
preds = bst.predict(dtrain)
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Training Numpy array precision:", precision_score(y_train, best_preds, average='macro'))

In [ ]:
preds = bst.predict(dtest)
# extracting most confident predictions
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Numpy array precision:", precision_score(y_test, best_preds, average='macro'))